In [1]:
import gdal
import rasterio
import numpy as np

In [13]:
filepath = r"../data/raw/satw/temp.tiff"
filepath2 = r"../data/raw/satw/la.tif"


# Open the file:
raster = gdal.Open(filepath2)

# Check type of the variable 'raster'
type(raster)

osgeo.gdal.Dataset

In [14]:
# Projection
print(raster.GetProjection())

# Dimensions
print( raster.RasterXSize )
print( raster.RasterYSize )

# Number of bands
print( raster.RasterCount )

# Metadata for the raster dataset
raster.GetMetadata()

PROJCS["WGS 84 / Pseudo-Mercator",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Mercator_1SP"],PARAMETER["central_meridian",0],PARAMETER["scale_factor",1],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["X",EAST],AXIS["Y",NORTH],EXTENSION["PROJ4","+proj=merc +a=6378137 +b=6378137 +lat_ts=0.0 +lon_0=0.0 +x_0=0.0 +y_0=0 +k=1.0 +units=m +nadgrids=@null +wktext +no_defs"],AUTHORITY["EPSG","3857"]]
4096
4096
4


{'AREA_OR_POINT': 'Area'}

#### Read Raster as Array

In [25]:
# Read raster data as numeric array from GDAL Dataset
rasterArray = raster.ReadAsArray()

In [26]:
type(rasterArray)


numpy.ndarray

In [53]:
print( rasterArray.strides, rasterArray.nbytes )
rasterArray.shape

rasterArray[0][0].size, rasterArray[0][0].size * 886 * 3




# Get nodata value from the GDAL band object
#nodata = band.GetNoDataValue()


(496160, 560, 1) 1488480


(560, 1488480)

#### Use Rasterio

In [5]:
import rasterio
import rasterio.features
import rasterio.warp

with rasterio.open('../data/raw/satw/temp.tiff') as dataset:

    # Read the dataset's valid data mask as a ndarray.
    mask = dataset.dataset_mask()

    # Extract feature shapes and values from the array.
    for geom, val in rasterio.features.shapes(
            mask, transform=dataset.transform):

        # Transform shapes from the dataset's own coordinate
        # reference system to CRS84 (EPSG:4326).
        geom = rasterio.warp.transform_geom(
            dataset.crs, 'EPSG:4326', geom, precision=6)

        # Print GeoJSON shapes to stdout.
        print(geom)

{'type': 'Polygon', 'coordinates': [[[-70.554252, 43.415611], [-70.554252, 41.468834], [-69.323784, 41.468834], [-69.323784, 43.415611], [-70.554252, 43.415611]]]}


#### Investigate Bands

In [30]:
# Read the raster band as separate variable
band = raster.GetRasterBand(1)

# Check type of the variable 'band'
print( type(band))

# Data type of the values
gdal.GetDataTypeName(band.DataType)

<class 'osgeo.gdal.Band'>


'Byte'

In [32]:
nodata = band.GetNoDataValue()
print(  type(nodata) )

<class 'NoneType'>


In [24]:
# # Compute statistics if needed
# if band.GetMinimum() is None or band.GetMaximum()is None:
#     band.ComputeStatistics(0)
#     print("Statistics computed.")

# # Fetch metadata for the band
# band.GetMetadata()

# # Print only selected metadata:
# print ("[ NO DATA VALUE ] = ", band.GetNoDataValue()) # none
print ("[ MIN ] = ", band.GetMinimum())
print ("[ MAX ] = ", band.GetMaximum())

[ MIN ] =  None
[ MAX ] =  None


#### rasterio work

In [15]:

# Read raster bands directly to Numpy arrays.
#
with rasterio.open('../data/raw/satw/la.tif') as src:
    r, g, b, o = src.read()

    # Combine arrays in place. Expecting that the sum will
    # temporarily exceed the 8-bit integer range, initialize it as
    # a 64-bit float (the numpy default) array. Adding other
    # arrays to it in-place converts those arrays "up" and
    # preserves the type of the total array.
    total = np.zeros(r.shape)
    for band in r, g, b:
        total += band
    total /= 3
    print(total)

    # Write the product as a raster band to a new 8-bit file. For
    # the new file's profile, we start with the meta attributes of
    # the source file, but then change the band count to 1, set the
    # dtype to uint8, and specify LZW compression.
    profile = src.profile
    profile.update(dtype=rasterio.uint8, count=1, compress='lzw')

    with rasterio.open('../data/raw/satw/la-gray.tiff', 'w', **profile) as dst:
        dst.write(total.astype(rasterio.uint8), 1)

[[ 80.33333333  99.33333333 105.66666667 ... 118.         105.33333333
  103.33333333]
 [ 97.33333333 113.         117.         ... 113.         105.
  105.        ]
 [128.33333333 134.66666667 132.         ... 111.         106.
  104.        ]
 ...
 [247.         247.         247.33333333 ...  44.          44.66666667
   47.66666667]
 [238.         239.33333333 240.         ...  45.66666667  46.
   46.33333333]
 [169.66666667 198.66666667 216.33333333 ...  46.66666667  44.66666667
   44.        ]]


### Get pixel value at coordinates

In [40]:
from osgeo import gdal

driver = gdal.GetDriverByName('GTiff')
filename = "../data/raw/satw/temp.tiff" #path to raster
raster = gdal.Open(filename)
band = (raster.GetRasterBand(1),raster.GetRasterBand(2),raster.GetRasterBand(3))

cols = raster.RasterXSize
rows = raster.RasterYSize

transform = raster.GetGeoTransform()
print( "transform --> ", transform )
xOrigin = transform[0]
yOrigin = transform[3]
pixelWidth = transform[1]
pixelHeight = -transform[5]

print( "cols, rows", cols, rows )
print(  pixelWidth, pixelHeight )
print( xOrigin, yOrigin )

data = (band[0].ReadAsArray(0, 0, cols, rows),
        band[1].ReadAsArray(0, 0, cols, rows),
         band[2].ReadAsArray(0, 0, cols, rows))


# points_list = [ (355278.165927, 4473095.13829), (355978.319525, 4472871.11636) ] #list of X,Y coordinates
points_list = [ (42.149770, -70.160992), (42.688064, -69.981054), (41.890654, -70.311888)] #list of X,Y coordinates

for point in points_list:
    col = int((point[1] - xOrigin) / pixelWidth)
    row = int((yOrigin - point[0] ) / pixelHeight)

    print(row,col)
    print( data[0][row][col], data[1][row][col], data[2][row][col] )

transform -->  (-70.55425241263127, 0.002197265625, 0.0, 43.41561113664346, 0.0, -0.002197265625)
cols, rows 560 886
0.002197265625 0.002197265625
-70.55425241263127 43.41561113664346
576 178
98 204 0
331 260
66 194 0
694 110
254 239 0


### call gdallocationinfo as python subprocess

In [28]:

import subprocess

bathymetry = "../data/raw/bathymetry/bathymetry.tif"
sst = "../data/raw/aquaMODIS-L2-sst/2016-07.tiff"


test = subprocess.Popen(["gdallocationinfo",
                        "-valonly",
                        "-wgs84",
                        sst,
                        "-70.164059",
                         "42.042833"
                        ], stdout=subprocess.PIPE)

output = test.communicate()
val = float(output[0].decode("utf-8"))
cel = ((val / 255) * 44) -2

print( cel )



25.26274509803922
